In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [271]:
# 데이터 로드
df_j = pd.read_csv("./datasets/raw/대주주외국인지분.csv", encoding="cp949")


In [273]:
# 중복값 삭제
df_j.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df_j.reset_index(drop=True, inplace=True)

In [274]:
# 외국계 기업 삭제
df_j = df_j[df_j["거래소코드"].astype("str").str[:0] != "9"]

In [275]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df_j[df_j["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df_j["회사명"].unique())
dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_joncat = df_j[df_j["회사명"]==i]
    df_joncat = df_joncat[df_joncat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_joncat)
df_j = pd.concat(dfs)
df_j.reset_index(drop=True, inplace=True)


In [276]:
# 회사명이 2개인 거래소코드 존재 확인 
list_overlap_code = list()
for i in list(df_j["거래소코드"].unique()):
    if len(df_j[df_j["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

[36420]

In [277]:
## 기업명 변경
df_j[df_j['거래소코드']=='036420']
df_j.loc[df_j['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [278]:
# 회사명이 2개인 거래소코드 존재 확인 - 엄슴
list_overlap_code = list()
for i in list(df_j["거래소코드"].unique()):
    if len(df_j[df_j["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

[]

In [279]:
df_j.isna().sum()

회사명             0
거래소코드           0
회계년도            0
보통주식수(주)        3
외국인_주식분포비율    337
dtype: int64

In [280]:
# 결측치 0으로 대체
df_j.fillna(0, axis=0,inplace=True)

In [281]:
df = df_j.copy()

In [282]:
# 대주주지분 변화 
aa = df.groupby(["회사명","회계년도"]).sum()[["보통주식수(주)"]].diff(1)

In [283]:
# 그룹바이 풀기
aa.reset_index(drop=False, inplace=True)

In [284]:
# 외국인 지분 변화분 계산
bb = df.groupby(["회사명","회계년도"]).sum()[["외국인_주식분포비율"]].diff(1)

In [285]:
# 그룹바이 풀기
bb.reset_index(drop=False, inplace=True)

In [286]:
# 데이터프레임으로 만들기
aa_1 = pd.DataFrame(aa["보통주식수(주)"])
bb_1 = pd.DataFrame(bb["외국인_주식분포비율"])

In [287]:
# 만든 데이터프레인 합치기
aabb = pd.concat([aa_1, bb_1], axis=1)

In [288]:
df = df[["회사명","거래소코드",	"회계년도"]]

In [289]:
df_jj = pd.concat([df,aabb], axis=1)

In [290]:
# 컬럼명 변경
df_jj.columns = ["회사명"	,"거래소코드",	"회계년도",	"대주주지분변화분",	"외국인지분분변화"]

In [291]:
# 거래소코드 패딩
df_jj["거래소코드"] = df_jj["거래소코드"].astype("str").str.rjust(6,"0")

In [295]:
df_jj.fillna(0, axis=0,inplace=True)

In [296]:
df_jj

,회사명,거래소코드,회계년도,대주주지분변화분,외국인지분분변화
0,(주)CMG제약,058820,2011/12,0.00,0.00
1,(주)CMG제약,058820,2012/12,0.00,0.22
2,(주)CMG제약,058820,2013/12,27.98,-0.04
3,(주)CMG제약,058820,2014/12,0.00,0.09
4,(주)CMG제약,058820,2015/12,-3.49,0.31
...,...,...,...,...,...
20909,흥아해운(주),003280,2017/12,-0.84,-0.12
20910,흥아해운(주),003280,2018/12,22.47,0.93
20911,흥아해운(주),003280,2019/12,-22.28,-4.82
20912,흥아해운(주),003280,2020/12,0.65,0.50


In [297]:
df_jj.to_csv("대주주외국인지분변화.csv", encoding= "utf-8-sig")